# Evaluation 

`advsecurenet` supports both benign and adversarial evaluation. In benign evaluation, the model is evaluated on clean images. In adversarial evaluation, there are multiple options to evaluate the model, the defense or the attack itself. Currently supported adversarial evaluations are: 

1. **Attack Success Rate**: The ratio of successful adversarial examples generated by the attack. Successful adversarial examples are those that are initially correctly classified by the model and are then misclassified after the attack. If the attack is untargeted, the misclassification should be to any class other than the original class. If the attack is targeted, the misclassification should be to the target class.

2. **Perturbation Distance**: The average perturbation distance between the original and adversarial examples. The perturbation distance is calculated as the Lp norm of the difference between the original and adversarial examples.

3. **Transferability**: The ratio of successful adversarial examples that fool both the model and a surrogate model.

4. **Robustness Gap**: The difference between the accuracy of the model on clean images and the accuracy of the model on clean images after applying the defense, such as adversarial training.

5. **Similarity**: The similarity between the original and adversarial examples. Supported metrics are SSIM and PSNR.

6. **Perturbation Efficiency**: The effectiveness score is the attack success rate divided by the perturbation distance. The higher the score, the more effective the attack.

There are two ways to adversarially evaluate the model using `advsecurenet` API. We can either utilize the `Attacker` or we can manually iterate over the dataset and calculate the evaluation metrics.

**Note:** `advsecurenet` CLI also supports evaluation. You can further use Distributed Evaluation to evaluate the model on multiple GPUs using the CLI.

## Adversarial Evaluation

### Using the Attacker

In [ ]:

from advsecurenet.attacks.gradient_based import FGSM, LOTS
from advsecurenet.shared.types.configs.attack_configs import FgsmAttackConfig, LotsAttackConfig
from advsecurenet.shared.types.configs.attack_configs.attacker_config import AttackerConfig
from advsecurenet.models.model_factory import ModelFactory
from advsecurenet.datasets.dataset_factory import DatasetFactory
from advsecurenet.attacks.attacker import Attacker
from advsecurenet.dataloader.data_loader_factory import DataLoaderFactory
from advsecurenet.shared.types.configs.preprocess_config import \
    PreprocessConfig, PreprocessStep
from advsecurenet.shared.types.configs.device_config import DeviceConfig
from advsecurenet.utils.adversarial_target_generator import AdversarialTargetGenerator
from advsecurenet.datasets.targeted_adv_dataset import AdversarialDataset
from advsecurenet.shared.adversarial_evaluators import adversarial_evaluators


In [ ]:
# Define the model
model = ModelFactory.create_model(model_name='resnet18', num_classes=10, pretrained=True)

In [ ]:
# Lets define the preprocessing configuration we want to use
preprocess_config = PreprocessConfig(
    steps=[
        PreprocessStep(name='Resize', params={'size': 32}),
        PreprocessStep(name='CenterCrop', params={'size': 32}),
        PreprocessStep(name='ToTensor'),
        PreprocessStep(name='ToDtype', params={'dtype': 'torch.float32', 'scale': True}),
        PreprocessStep(name='Normalize', params={'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}),
    ]
)

# Define the dataset
dataset = DatasetFactory.create_dataset(dataset_type="cifar10", preprocess_config=preprocess_config, return_loaded=False)
test_data = dataset.load_dataset(train=False)

In [ ]:
# Define the dataloder
dataloader = DataLoaderFactory.create_dataloader(dataset=test_data, batch_size=32 )

In [ ]:
# define the device config
device = DeviceConfig(processor="mps")

# Define the fgsm config
fgsm_config = FgsmAttackConfig(
    targeted=False,
    epsilon=0.1,
    device=device,
)

# Now we can define the attack
attack = FGSM(config=fgsm_config)  

In [ ]:
# lets check the evaluators 
print(adversarial_evaluators.keys())


In [ ]:
evaluators = ["attack_success_rate", "robustness_gap","perturbation_distance","similarity" ]
attacker_config = AttackerConfig(
    model=model,
    attack= attack,
    dataloader=dataloader,
    device=device,
    return_adversarial_images=False,
    evaluators=evaluators,
)

attacker = Attacker(config=attacker_config)

attacker.execute()

In [ ]:
# We can also evaluate the transferability of the adversarial examples

# define a target model
vgg16 = ModelFactory.create_model(model_name='vgg16', num_classes=10, pretrained=False)

evaluators = ["transferability"]
attacker_config = AttackerConfig(
    model=model,
    attack= attack,
    dataloader=dataloader,
    device=device,
    return_adversarial_images=False,
    evaluators=evaluators,
)

attacker = Attacker(config=attacker_config, target_models=[vgg16])

attacker.execute()

### Using the Evaluator

If you don't want to use the `Attacker` class, you can manually iterate over the dataset and calculate the evaluation metrics. Here is an example of how to do that:


In [ ]:
from advsecurenet.evaluation.adversarial_evaluator import AdversarialEvaluator

In [ ]:

with AdversarialEvaluator(evaluators=["attack_success_rate"]) as evaluator:
    for images, labels in dataloader:
        model.eval()
        model = model.to(device.processor)
        images = images.to(device.processor)
        labels = labels.to(device.processor)
        adv_images = attack.attack(model, images, labels)
        evaluator.update(model=model, original_images=images, true_labels=labels, adversarial_images=adv_images, is_targeted=False)
        
print(evaluator.get_results())

## Benign Evaluation

You can also evaluate the model on clean images using the `Tester` class. Here is an example of how to do that:

In [ ]:
from advsecurenet.shared.types.configs.test_config import TestConfig
from advsecurenet.evaluation.tester import Tester


In [ ]:
test_config = TestConfig(
    model=model,
    test_loader=dataloader,
    processor=device.processor,
    topk=3 # You can change this to any value you want
)

tester = Tester(config=test_config)

tester.test()